MARKDOWN CELL:
# 🚪🚪🚪 The Monty Hall Problem: Frequentist Simulation vs. Bayesian Analysis

This notebook tackles the classic **Monty Hall Problem** to demonstrate two fundamental, yet complementary, approaches to probability and inference:

* **Frequentist (Monte Carlo Simulation)**: Estimating probabilities through the long-run relative frequency of outcomes in repeated, independent trials (simulations).
* **Bayesian (Bayes' Theorem)**: Updating our initial belief (**prior**) about an outcome based on new evidence (**likelihood**) to form a refined belief (**posterior**).

## 📌 Classical Monty Hall Setup

1.  **Three Doors**: One car (the prize) and two goats are placed randomly behind doors 1, 2, and 3
2.  **Player's Initial Choice**: The player picks one door, $\mathbf{D}_{\text{pick}}$, uniformly at random (could have chosen any one)
3.  **Host's Action**: The host (who knows where the car is) opens one of the *other* two doors, $D_{open}$, revealing a goat
4.  **The Offer**: The host always offers the player the choice to **stay** with their initial pick or **switch** to the other remaining closed door, $D_{switch}$
5.  **Host's Rules (Crucial)**:
    * The host must always open one of the doors the player did not pick
    * The host must always open a door with a goat (not the one with the car)
    * If the host has two goat doors to choose from (i.e. the player's initial pick was the car), the host chooses which goat door to open uniformly at random (50/50)

## 💡 Learning Goals: Frequentist vs. Bayesian

By the end of this exercise, you will be able to:

* **Frequentist Insight**: Simulate a process many times to empirically estimate the true underlying probability, demonstrating how **relative frequencies** converge to the **theoretical probability** (Law of Large Numbers)
* **Bayesian Insight**: Apply **Bayes' theorem** to formally update an initial **prior belief** ($P(Car)$) using the new evidence provided by the host's action ($P(Host\text{ opens } Goat)$) to calculate the refined **posterior probability** ($P(Car \mid Host\text{ opens } Goat)$)
* **Conceptual Clarity**: Explain *why* switching doors is the optimal strategy and demonstrate how both paradigms arrive at the same conclusion: **2/3 probability of winning by switching**

## Setup

Run the following cell to import libraries and set a reproducible random seed


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
np.random.seed(42)  # reproducibility

In [ ]:
# Simulate the Monty Hall problem
def simulate_monty_hall(n_trials: int) -> dict:
    """Simulate the classical Monty Hall problem for n_trials.
    
    Assumptions
    - Car is placed uniformly at random behind one of three doors
    - Player picks a door uniformly at random
    - Host knows the car location, opens a different door that always has a goat
    - If host has a choice of two goat doors, he picks one uniformly at random
    - Player either always stays or always switches
    
    Returns counts of wins for stay and for switch
    """
    # Doors encoded as 0,1,2
    car_doors = np.random.randint(0, 3, size=n_trials)
    picks = np.random.randint(0, 3, size=n_trials)
    
    # Host opens a goat door that is not the player's pick
    host_opens = np.empty(n_trials, dtype=int)
    for i in range(n_trials):
        available = {0, 1, 2} - {picks[i]}
        # remove the car door from available if it is not the player's pick
        if car_doors[i] != picks[i]:
            available = available - {car_doors[i]}
        host_opens[i] = np.random.choice(list(available))
    
    # The remaining closed door after host opens is:
    remaining = 3 - picks - host_opens  # because 0+1+2 = 3 for doors 0,1,2
    
    stay_wins = np.sum(picks == car_doors)
    switch_wins = np.sum(remaining == car_doors)
    return {
            "stay_wins": int(stay_wins), 
            "switch_wins": int(switch_wins), 
            "n_trials": int(n_trials)
        }


## Part 1: 📊 Frequentist Simulation (Monte Carlo)

The **Frequentist** approach estimates the probability of an event by running the experiment repeatedly and calculating the **relative frequency** of that event. The Law of Large Numbers dictates that as the number of trials ($N$) grows, the empirical relative frequency converges to the true theoretical probability.

We expect:
* **Stay Probability**: $1/3$ (since the initial choice is random among three doors).
* **Switch Probability**: $2/3$ (since the probability that the car was *not* behind your initial pick is $2/3$, and the host's action transfers that $2/3$ probability to the remaining door).

In [ ]:
# Show the final results in a bar chart, comparing empirical rates to theoretical
N = 10
final_res = simulate_monty_hall(N)

# Calculate empirical rates
stay_rate_empirical = final_res['stay_wins'] / N
switch_rate_empirical = final_res['switch_wins'] / N

# Theoretical rates
stay_rate_theoretical = 1/3
switch_rate_theoretical = 2/3

# Data for plotting
strategies = ['Stay (Empirical)', 'Stay (Theoretical)', 'Switch (Empirical)', 'Switch (Theoretical)']
rates = [stay_rate_empirical, stay_rate_theoretical, switch_rate_empirical, switch_rate_theoretical]
colors = ['#4C72B0', '#A9A9A9', '#C44E52', '#A9A9A9'] # Blue for Stay, Red for Switch, Grey for Theoretical

plt.figure(figsize=(9, 6))
bars = plt.bar(strategies, rates, color=colors)

# Add value labels on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f'{yval:.3f}', ha='center', va='bottom', fontweight='bold')

plt.ylim(0, 1.0) # Set y-limit to 0-1 for probability scale
plt.ylabel('Win Probability')
plt.title(f'Monty Hall Problem: Final Win Rates After {N} Trials (Frequentist View)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Run a large batch and compute running means
N = 100
res = simulate_monty_hall(N) # res is a dict

# Re-simulate step by step to get running means for both strategies
stay_wins = 0 # of wins when staying
switch_wins = 0 # of wins when switching
running_stay = np.empty(N) # .empty method for efficiency
running_switch = np.empty(N)

# We need to iterate to capture the path of convergence
car_doors = np.random.randint(0, 3, size=N) # car locations
picks = np.random.randint(0, 3, size=N) # player picks
host_opens = np.empty(N, dtype=int) # host opens a door
for i in range(N):
    available = {0, 1, 2} - {picks[i]}
    if car_doors[i] != picks[i]:
        available = available - {car_doors[i]}
    host_opens[i] = np.random.choice(list(available))
    remaining = 3 - picks[i] - host_opens[i]
    stay_wins += int(picks[i] == car_doors[i])
    switch_wins += int(remaining == car_doors[i])
    running_stay[i] = stay_wins / (i + 1)
    running_switch[i] = switch_wins / (i + 1)

print(f"Total trials: {N}")
print(f"Stay strategy final win rate: {res['stay_wins'] / N:.4f} (Expected: 0.3333)")
print(f"Switch strategy final win rate: {res['switch_wins'] / N:.4f} (Expected: 0.6667)")

# Plot convergence of running win rates
plt.figure(figsize=(8, 5))
plt.plot(running_stay, label='Stay win rate')
plt.plot(running_switch, label='Switch win rate')
plt.axhline(1/3, linestyle='--', label='1/3 reference')
plt.axhline(2/3, linestyle='--', label='2/3 reference')
plt.xlabel('Trials')
plt.ylabel('Empirical probability')
plt.title('Monty Hall win rates converge to theoretical values')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

# Plot convergence of running win rates, zoomed into the first 100 trials
plt.figure(figsize=(10, 6))
# Using colors consistent with standard plot practices (blue for Stay, orange for Switch based on the plot image)
plt.plot(running_stay, label='Empirical Stay Win Rate', color='tab:blue', alpha=0.8)
plt.plot(running_switch, label='Empirical Switch Win Rate', color='tab:orange', alpha=0.8)

# Applying the colors to the reference lines for clarity
plt.axhline(1/3, linestyle='--', color='tab:blue', label='Theoretical 1/3 (Stay)')
plt.axhline(2/3, linestyle='--', color='tab:orange', label='Theoretical 2/3 (Switch)')

# Zooming into the first 100 trials
plt.xlim(0, 100) 

plt.xlabel('Number of Trials (N)', fontsize=12)
plt.ylabel('Empirical Probability (Relative Frequency)', fontsize=12)
plt.title('Convergence of Monty Hall Win Rates (Zoomed to 100 Trials)', fontsize=14, fontweight='bold')
plt.legend(loc='center right')
plt.grid(True, linestyle=':', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate across increasing batch sizes to show approximation quality
batch_sizes = [100, 1000, 10000, 100000, 200000]
summary = []
for n in batch_sizes:
    r = simulate_monty_hall(n)
    summary.append((n, r['stay_wins'] / n, r['switch_wins'] / n))
import pandas as pd
df = pd.DataFrame(summary, columns=['trials', 'stay_rate', 'switch_rate'])
df


## Part 2 - Bayesian analysis

We compute the posterior probability that your initially chosen door has the car after the host opens a different door to reveal a goat

Let
- C be the random variable for the door with the car
- P be the door you initially pick
- H be the event that the host opens a specific goat door

Assumptions
- Prior P(C = d) = 1/3 for d in {1,2,3}
- P is chosen uniformly at random and independent of C
- The host always opens a goat door different from P, and if both remaining doors have goats he selects randomly

By Bayes theorem
\[ P(C = c \mid H, P) = \frac{P(H \mid C = c, P) P(C = c)}{\sum_{c'} P(H \mid C = c', P) P(C = c')} \]

Fix an example: you pick door 1, and the host opens door 3 revealing a goat. Then

- If C = 1, host is forced to open door 2 or 3 with equal probability, so P(H = open 3 \mid C = 1, P = 1) = 1/2
- If C = 2, host is forced to open door 3, so P(H = open 3 \mid C = 2, P = 1) = 1
- If C = 3, that would reveal the car, so P(H = open 3 \mid C = 3, P = 1) = 0

With prior P(C = c) = 1/3, the normalized posteriors are

- P(C = 1 \mid H, P) proportional to (1/2) · (1/3) = 1/6
- P(C = 2 \mid H, P) proportional to 1 · (1/3) = 1/3
- P(C = 3 \mid H, P) proportional to 0 · (1/3) = 0

After normalization we get P(C = 1 \mid H, P) = 1/3 and P(C = 2 \mid H, P) = 2/3. So switching to door 2 wins with probability 2/3


In [ ]:
# Compute the posterior numerically for the example P=1, H=open 3
priors = np.array([1/3, 1/3, 1/3])  # for C=1,2,3

# Likelihoods P(H=open3 | C=c, P=1)
likelihoods = np.array([0.5, 1.0, 0.0])

unnormalized = priors * likelihoods
posterior = unnormalized / unnormalized.sum()
print('Posterior P(C | P=1, H=open 3) =', posterior)
print('Probability that switching to door 2 wins =', posterior[1])


In [ ]:
# Verify by enumerating equally likely worlds for P=door 0 and H=open door 2
doors = [0, 1, 2]  # 0-based indices for doors 1, 2, 3
prior = 1/3

num = 0.0
den = 0.0
for c in doors:         # car position
    p = 0               # player picks door 0
    # Likelihood P(H=open2 | C=c, P=0)
    if c == p:
        # Both remaining doors have goats, host chooses randomly, so 1/2 chance of opening door 2
        lh = 0.5
    elif c == 1:
        # Car at door 1, host must open door 2
        lh = 1.0
    else:
        # Car at door 2, host cannot open door 2
        lh = 0.0
    den += prior * lh
    # Switching wins iff car is behind the other closed door, which is door 1 in this event
    if c == 1:
        num += prior * lh

posterior_switch = num / den
print('Posterior probability that switching wins (by enumeration) =', posterior_switch)


## Extensions for exploration

Try the following
- Change the host policy so he sometimes does not offer a switch and observe how the simulation win rates change
- Generalize to N doors and verify that switching wins with probability (N − 1)/N
- Replace the uniform prior on the car door with a non-uniform prior and recompute the Bayes posterior


## References

- Monty Hall problem, classical assumptions and solutions, Wikipedia https://en.wikipedia.org/wiki/Monty_Hall_problem
- Stanford CS109 notes, Conditional Probability and Bayes theorem https://web.stanford.edu/class/archive/cs/cs109/cs109.1246/lectures/04_cond_bayes_annotated.pdf
- Wired article with a Python based simulation and discussion of convergence https://www.wired.com/story/monty-hall-problem-python
